# Openning Data

In [18]:
using SparseArrays
using JuMP
using Ipopt
using GLPK
using LinearAlgebra
using MAT

In [19]:
file = matopen("R5T3.mat")
varnames = names(file)

4-element Array{String,1}:
 "K"
 "L"
 "S"
 "U"

In [20]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U")
K = read(file, "K")

40

In [21]:
(m,n)=size(S)
(r,c) = size(L)

(28301, 200)

# constructing model

In [22]:
mzeros = zeros(m,1);

In [23]:
function opt_vector0(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector0 (generic function with 1 method)

In [24]:
function opt_vector(S,l,u,z,lambda)
    model = Model(Ipopt.Optimizer)
    
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @variable(model,b[1:m])
    
    # @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    
    @constraint(model, b .>= S*v)
    @constraint(model, b .>= -S*v)
    
    
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r) + lambda*sum(sqrt(1+b[j]) for j=1:m))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector (generic function with 1 method)

In [25]:
function opt_vector(S,l,u,z,lambda)
    model = Model(Ipopt.Optimizer)
    
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @variable(model,b[1:m])
    
    # @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    
    @constraint(model, b .>= S*v)
    @constraint(model, b .>= -S*v)
    
    
    @NLobjective(model,Min,sum(sqrt(1+z[i]+a[i]) for i=1:r) + lambda*sum(b[j] for j=1:m))
    optimize!(model)
    
    return JuMP.value.(v)
end

opt_vector (generic function with 1 method)

In [35]:
function zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-5
            indexes[k]= i
            k+=1
        end
    end
    return length(indexes[1:k-1])
end

zero_rows (generic function with 1 method)

In [27]:
function row_squares(V)
    r,c = size(V,1), size(V,2)
    squares = zeros(r,1)
    for i in 1:r
        z=0
        for j in 1:c
           z += (V[i,j])^2
        end
        #squares[i]=sqrt(z)
        squares[i]=z
    end
    return squares
end

row_squares (generic function with 2 methods)

In [28]:
function row_squares(V,z)
    r,c = size(V,1), size(V,2)
    for i in 1:r
        z[i]=z[i]+V[i,c]^2
    end
    return z
end

row_squares (generic function with 2 methods)

In [51]:
# we tried lambda = 100,150,200,250 and 200 had the best result among these choices
lambda = 200

200

In [52]:
l = L[:,1]
u = U[:,1]
z = ones(r,1)
v = opt_vector(S,l,u,z,lambda)
zero_rows(v)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   105344
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45739
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1800
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

27773

In [53]:
V=v;

In [54]:
z = zeros(r,1)
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V,z)
    v = opt_vector(S,l,u,z,lambda)
    #v = opt_vector0(S,l,u,z)
    V = [V v]
    print("**i : ",i,"\n")
    print("***",zero_rows(V),"\n") 
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   107820
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45973
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2034
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   106960
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45905
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1966
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   106280
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45767
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1828
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   107550
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45977
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2038
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   107564
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45960
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2021
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

...

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   102908
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45552
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1613
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   107006
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45886
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1947
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   108006
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45958
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2019
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   107930
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45967
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2028
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   103014
Number of nonzeros in Lagrangian Hessian.............:    28301

Total number of variables............................:    45577
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1638
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:    87878
        inequality constraints with only lower bounds:    87878
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [55]:
V

28301×200 Array{Float64,2}:
  0.788477    -0.912853  0.0200934  …  6.12226e-14  0.939489  -0.924096
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
 -1.82278e-8   0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0        …  0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0        …  0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  0.0          0.0       0.0           0.0          0.0        0.0
  ⋮                                 ⋱                         
  0.0          0.0       0.0     

In [56]:
n1 = zero_rows(V)
n2 = zero_rows((S*V)')
n1 ,n2

(25074, 200)

# writing to output file

In [58]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, V,",")
end